In [ ]:
# FF & Chrome Driver import by selenium
from selenium import webdriver

# Launch Firefox 
driver = webdriver.Firefox()

# Set Dynamic Result Page template
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

# SET THE KEYWORD HERE!!! Sets the dynamic search_term to put inside {} in 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
url = get_url('mini fan')

# Activates the search
driver.get(url)

#Prepare Beautiful Soup and select through "s-search-result"
import csv
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type': 's-search-result'})

# Extract TITLE, ASIN, STARS, RATINGS, URL
def extract_record(item):

    atag = item.h2.a
    title = atag.text.strip()
    asin = item.get('data-asin')
    Url1 = 'https://www.amazon.com' + atag.get('href')

    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return

    try:
        stars = item.i.text
        ratings_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        stars = ''
        ratings_count = ''
    
    result = (title, asin, price, stars, ratings_count, Url1)

    return result
# Creating an array 
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})                            
for item in results:
    record = extract_record(item)
    if record:                             
            records.append(record)

# Write CSV            
    with open('test.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'ASIN', 'Price', 'Stars', 'RatingsCount', 'URL'])
        writer.writerows(records)